In [1]:
from llms.llm_provider_factory import LLMProviderFactory
from config.config import config
from config.settings import settings
from loaders.pdf_loader import PDFLoader
from processors.text_processor import TextProcessor
from vector_dbs.providers.vector_store import VectorStore
from utils.logger import get_logger
from utils.exceptions import PDFLoadError
from llms.rag_provider import RAGProvider
from llms.templates.template_parser import TemplateParser
from mongo_db.chat_log_manager import ChatLogManager

/home/ahmadhammad/Desaisiv-AI-Agent/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from motor.motor_asyncio import AsyncIOMotorClient

mongo_conn = AsyncIOMotorClient(config.MONGODB_URL)
db_client = mongo_conn[config.MONGODB_DATABASE]

In [3]:
chat_log_manager = ChatLogManager(
    mongo_conn=mongo_conn, db_client=db_client, collection_name=config.MONGODB_COLLECTION
)

In [18]:
r = await chat_log_manager.get_chat_history(user_id="1", session_id="2")

In [19]:
r

[{'role': 'system',
  'content': 'أنت  سالم من ديسايسيف مساعد لتوليد استجابة للمستخدم.\nسيتم تزويدك بمجموعة من الوثائق المرتبطة باستفسار المستخدم.\nتجاهل الوثائق التي ليست ذات صلة باستفسار المستخدم.\nيمكنك الاعتذار للمستخدم إذا لم تتمكن من توليد استجابة.\nأجب بالإنجليزية إذا كان السؤال باللغة الإنجليزية، وباللهجة السعودية إذا كان السؤال باللغة العربية.\nكن مهذبًا ومحترمًا مع المستخدم.\nكن دقيقًا ومختصرًا في إجابتك. تجنب المعلومات غير الضرورية.'},
 {'role': 'user',
  'content': "## الوثيقة رقم: 1\n### المحتوى: Contact, Support or Inquires \nIf you have any questions support, please contact \nMobile number: 00966570745585 \nEmail Address: info@desaisiv.com\n## الوثيقة رقم: 2\n### المحتوى: o Reduce manual data entry and errors, allowing underwriters to focus on risk assessment. \no Improve efficiency in policy pricing, claim evaluation, and overall decision-making for \ninsurers. \n \nRequired Files for Feature Usage \nTo utilize the platform’s features effectively, the following files ma

In [3]:
# logger.info("Loading PDFs...")
pdf_loader = PDFLoader(settings.PDF_DIRECTORY)
pdf_docs = pdf_loader.load_pdfs()
if not pdf_docs:
    raise PDFLoadError("No PDFs found in the directory.")

llm_provider_factory = LLMProviderFactory(config, settings)
embedding_client = llm_provider_factory.create(provider=settings.EMBEDDING_BACKEND)
embedding_client.set_embedding_model(model_id=settings.EMBEDDING_MODEL_ID,
                                        embedding_size=settings.EMBEDDING_MODEL_SIZE)

# logger.info("Splitting text into chunks...")
text_processor = TextProcessor(settings.CHUNK_SIZE, settings.CHUNK_OVERLAP)
documents = text_processor.split_documents(pdf_docs)

In [4]:
r = embedding_client.embed_text(["gello", "ahmad"])

In [5]:
len(r)

768

In [6]:
vector_store = VectorStore()
vector_store.build_vector_store(documents, embedding_client.embed_text)
vector_store.save_vector_store(settings.VECTOR_STORE_PATH)

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


[[0.0708647072315216, -0.056666068732738495, 0.05287548154592514, 0.02913612499833107, -0.06023178994655609, 0.03287852555513382, -0.027754809707403183, 0.05062682926654816, -0.008207585662603378, -0.0011624735780060291, 0.032107558101415634, -0.005167887546122074, -0.005256227683275938, -0.029585855081677437, 0.04127885401248932, -0.031192036345601082, 0.0014857174828648567, 0.01218288391828537, -0.020093325525522232, -0.01592528633773327, 0.03472563624382019, 0.04478032886981964, -0.03372980281710625, -0.032717909663915634, 0.06003905087709427, -0.028172416612505913, 0.03797012194991112, -0.0720854103565216, 0.01896899938583374, -0.08063028752803802, 0.0305977500975132, -0.03527173772454262, 0.0020438653882592916, -0.031577520072460175, -0.030324699357151985, 0.0001880988565972075, 0.02094460092484951, 0.03462926298379898, 0.03771313279867172, 0.0016393085243180394, -0.005264258477836847, 0.0005395764601416886, 0.026630481705069542, 0.0028128246776759624, -0.01700945757329464, 0.0729

In [8]:
vector_store = VectorStore()
vector_store.load_vector_store(settings.VECTOR_STORE_PATH, embedding_client.embed_text)

NameError: name 'embedding_client' is not defined

In [14]:
q = "Insurance Pricing Policy"
result = vector_store.query(q, k=2)

In [15]:
result

[(Document(id='238c1e8d-8ef9-435f-b238-49663213fde3', metadata={}, page_content='Insurance Pricing Policy \n• Under 18 years old → 1,000 SR \n• Ages 19 - 30 → 1,200 SR \n• Ages 31 - 65 → 1,500 SR \n• Over 65 years old → 2,000 SR'),
  np.float32(0.71244025))]

In [9]:
query_vector = embedding_client.embed_text(q)  # If you have an embedding function
vector_store.vector_db.asimilarity_search(q)

<coroutine object FAISS.asimilarity_search at 0x7f72e1573400>

In [14]:
query_vector[1]

IndexError: list index out of range

In [2]:
settings.PRIMARY_LANG

'ar'

In [3]:
settings.DEFAULT_LANG

'en'

In [9]:
template_parser = TemplateParser(language=settings.PRIMARY_LANG, default_language=settings.DEFAULT_LANG)
llm_provider_factory = LLMProviderFactory(config, settings)

In [10]:
generation_client = llm_provider_factory.create(provider=settings.GENERATION_BACKEND)
generation_client.set_generation_model(model_id=settings.GENERATION_MODEL_ID)

embedding_client = llm_provider_factory.create(provider=settings.EMBEDDING_BACKEND)
embedding_client.set_embedding_model(model_id=settings.EMBEDDING_MODEL_ID,
                                            embedding_size=settings.EMBEDDING_MODEL_SIZE)

vector_store = VectorStore()
vector_store.load_vector_store(settings.VECTOR_STORE_PATH, embedding_client.embed_text)

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [11]:
rag_object = RAGProvider(vectordb_client=vector_store, generation_client=generation_client,
                         embedding_client=embedding_client, template_parser=template_parser)

In [12]:
rag_object.template_parser.default_language

'en'

In [14]:
generation_client.generate_text("What's my name?", chat_history=[{"role": "user", "content": "my is name is ahmad"}])

'Your name is Ahmad.'

In [13]:
r = rag_object.answer_rag_question("كيف ممكن تساعدني ديسايسيف في التامين")

In [17]:
print(r[1])

## الوثيقة رقم: 1
### المحتوى: smarter, more efficient, and customer-centric insurance solutions. By collaborating with top machine 
learning scientists from the University of Oxford and accessing an extensive database of patient records, 
Desaisiv has achieved an impressive average prediction accuracy of over 95%.  
  
Desaisiv Ledger (bucket) 
Desaisiv has 1 ledge total of insurance claims of over 1 billion SAR. 
محفظة تأمينية تتجاوز المليار ريال سعودي لشركة تكنولوجيا التأمين ديسايسف 
 
 
General Features 
Desaisiv's platform offers a range of features designed to streamline health insurance processes for 
insurers and third-party administrators (TPAs): 
• Effortless Data Management: Simplifies the input and maintenance of employee lists, claim 
experiences (CE), and interaction data, reducing the need for manual data entry.  
Advanced OCR Technology: Utilizes Optical Character Recognition to extract data from PDFs, 
ensuring accuracy and efficiency in data processing.
## الوثيقة رقم

In [13]:
print(r[2])

[{'role': 'system', 'content': "You are an assistant to generate a response for the user.\nYou will be provided by a set of docuemnts associated with the user's query.\nIgnore the documents that are not relevant to the user's query.\nYou can applogize to the user if you are not able to generate a response.\nYou have to generate response in the same language as the user's query.\nBe polite and respectful to the user.\nBe precise and concise in your response. Avoid unnecessary information."}, {'role': 'user', 'content': "## Document No: 1\n### Content: Contact, Support or Inquires \nIf you have any questions support, please contact \nMobile number: 00966570745585 \nEmail Address: info@desaisiv.com\n## Document No: 2\n### Content: Who is Desaisiv? \nDesaisiv is an insurtech company specializing in health insurance management through AI-powered \nsolutions. Founded in 2022 by serial entrepreneurs Saed Khawaldeh and Mohamad Nabhan, the \ncompany operates primarily in the Middle East and Nor

In [20]:
r = rag_object.answer_rag_question("What's my name?")

In [21]:
print(r[0])

I'm sorry, but I don't have access to your name. If you have any other questions or need assistance, feel free to ask!


In [22]:
r[2][1]

{'role': 'user',
 'content': "## Document No: 1\n### Content: Contact, Support or Inquires \nIf you have any questions support, please contact \nMobile number: 00966570745585 \nEmail Address: info@desaisiv.com\n## Document No: 2\n### Content: Advanced OCR Technology: Utilizes Optical Character Recognition to extract data from PDFs, \nensuring accuracy and efficiency in data processing.  \n• AI-Powered Analytics: Provides comprehensive insights into health insurance consumption, \nincluding employee utilization patterns, preferred healthcare providers, and types of medical \nvisits.  \n• Future Cost Predictions: Offers predictive analysis to anticipate future health insurance costs, \naiding in effective budgeting and planning.  \n• Customization and Optimization: Allows for tailoring of health insurance benefits to align with \norganizational needs and budgets, either manually or through an optimization algorithm.\n## Document No: 3\n### Content: Insurance Pricing Policy \n• Under 18 y

In [18]:
len(r[2])

2

In [16]:
print(r[2])

[{'role': 'system', 'content': "You are an assistant to generate a response for the user.\nYou will be provided by a set of docuemnts associated with the user's query.\nIgnore the documents that are not relevant to the user's query.\nYou can applogize to the user if you are not able to generate a response.\nYou have to generate response in the same language as the user's query.\nBe polite and respectful to the user.\nBe precise and concise in your response. Avoid unnecessary information."}, {'role': 'user', 'content': "## Document No: 1\n### Content: Insurance Pricing Policy \n• Under 18 years old → 1,000 SR \n• Ages 19 - 30 → 1,200 SR \n• Ages 31 - 65 → 1,500 SR \n• Over 65 years old → 2,000 SR\n## Document No: 2\n### Content: o Reduce manual data entry and errors, allowing underwriters to focus on risk assessment. \no Improve efficiency in policy pricing, claim evaluation, and overall decision-making for \ninsurers. \n \nRequired Files for Feature Usage \nTo utilize the platform’s fe

In [12]:
r[2]

[{'role': 'system',
  'content': "You are an assistant to generate a response for the user.\nYou will be provided by a set of docuemnts associated with the user's query.\nIgnore the documents that are not relevant to the user's query.\nYou can applogize to the user if you are not able to generate a response.\nYou have to generate response in the same language as the user's query.\nBe polite and respectful to the user.\nBe precise and concise in your response. Avoid unnecessary information."},
 {'role': 'user',
  'content': "## Document No: 1\n### Content: o Enable insurers to revise offers, negotiate terms, and finalize agreements with clients \nseamlessly. \no Streamline the quotation process, reducing manual effort and improving transparency \nbetween all parties. \n• Provider Comparison \no Compare provider networks across different insurance classes and tiers to assess \ncoverage quality. \no Upload network lists and evaluate them against competitors to identify the best options. \

In [ ]:
generation_client.generate_text("Hello")

In [8]:
messages = [ 
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": "Hello my name is Ahmad Hammad, how is your day"},
    # {"role": "user", "content": "What was my name?"},
]

In [ ]:
result = generation_client.generate_response(messages=messages)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [13]:
print(result)

[INST] <<SYS>>
You are a helpful AI assistant.
<</SYS>>

Hello my name is Ahmad Hammad, how is your day [/INST] Hello Ahmad Hammad! As an AI, I don't have feelings, but I'm here to help you. How can I assist you today? 


`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [13]:
vector_store.vector_db

In [10]:
vector_store.vector_db.similarity_search_with_score(query=q)

TypeError: 'list' object is not callable

In [ ]:
result

[(Document(id='7586c809-c8ed-4e41-a91b-f5e5242df885', metadata={}, page_content='o Enable insurers to revise offers, negotiate terms, and finalize agreements with clients \nseamlessly. \no Streamline the quotation process, reducing manual effort and improving transparency \nbetween all parties. \n• Provider Comparison \no Compare provider networks across different insurance classes and tiers to assess \ncoverage quality. \no Upload network lists and evaluate them against competitors to identify the best options. \no Ensure policyholders receive access to top-tier healthcare providers while optimizing \ncosts. \n• Underwriter Tool \no Automate the conversion of PDF documents into structured Excel files for underwriting \npurposes.'),
  np.float32(6.3113356)),
 (Document(id='60030367-f8b3-401e-9d69-c9aff4f08a53', metadata={}, page_content='o Reduce manual data entry and errors, allowing underwriters to focus on risk assessment. \no Improve efficiency in policy pricing, claim evaluation, 

In [14]:
result[0][0].page_content

'o Enable insurers to revise offers, negotiate terms, and finalize agreements with clients \nseamlessly. \no Streamline the quotation process, reducing manual effort and improving transparency \nbetween all parties. \n• Provider Comparison \no Compare provider networks across different insurance classes and tiers to assess \ncoverage quality. \no Upload network lists and evaluate them against competitors to identify the best options. \no Ensure policyholders receive access to top-tier healthcare providers while optimizing \ncosts. \n• Underwriter Tool \no Automate the conversion of PDF documents into structured Excel files for underwriting \npurposes.'

In [15]:
result

[(Document(id='7586c809-c8ed-4e41-a91b-f5e5242df885', metadata={}, page_content='o Enable insurers to revise offers, negotiate terms, and finalize agreements with clients \nseamlessly. \no Streamline the quotation process, reducing manual effort and improving transparency \nbetween all parties. \n• Provider Comparison \no Compare provider networks across different insurance classes and tiers to assess \ncoverage quality. \no Upload network lists and evaluate them against competitors to identify the best options. \no Ensure policyholders receive access to top-tier healthcare providers while optimizing \ncosts. \n• Underwriter Tool \no Automate the conversion of PDF documents into structured Excel files for underwriting \npurposes.'),
  np.float32(5.7243996)),
 (Document(id='60030367-f8b3-401e-9d69-c9aff4f08a53', metadata={}, page_content='o Reduce manual data entry and errors, allowing underwriters to focus on risk assessment. \no Improve efficiency in policy pricing, claim evaluation, 

In [19]:
result.page

[Document(id='8e557090-ebb2-4dbc-bc2c-ad1e2774a623', metadata={}, page_content="smarter, more efficient, and customer-centric insurance solutions. By collaborating with top machine \nlearning scientists from the University of Oxford and accessing an extensive database of patient records, \nDesaisiv has achieved an impressive average prediction accuracy of over 95%.  \n  \nDesaisiv Ledger (bucket) \nDesaisiv has 1 ledge total of insurance claims of over 1 billion SAR. \nمحفظة تأمينية تتجاوز المليار ريال سعودي لشركة تكنولوجيا التأمين ديسايسف \n \n \nGeneral Features \nDesaisiv's platform offers a range of features designed to streamline health insurance processes for \ninsurers and third-party administrators (TPAs): \n• Effortless Data Management: Simplifies the input and maintenance of employee lists, claim \nexperiences (CE), and interaction data, reducing the need for manual data entry.  \nAdvanced OCR Technology: Utilizes Optical Character Recognition to extract data from PDFs, \nens

In [18]:
print(result[0].page_content)

smarter, more efficient, and customer-centric insurance solutions. By collaborating with top machine 
learning scientists from the University of Oxford and accessing an extensive database of patient records, 
Desaisiv has achieved an impressive average prediction accuracy of over 95%.  
  
Desaisiv Ledger (bucket) 
Desaisiv has 1 ledge total of insurance claims of over 1 billion SAR. 
محفظة تأمينية تتجاوز المليار ريال سعودي لشركة تكنولوجيا التأمين ديسايسف 
 
 
General Features 
Desaisiv's platform offers a range of features designed to streamline health insurance processes for 
insurers and third-party administrators (TPAs): 
• Effortless Data Management: Simplifies the input and maintenance of employee lists, claim 
experiences (CE), and interaction data, reducing the need for manual data entry.  
Advanced OCR Technology: Utilizes Optical Character Recognition to extract data from PDFs, 
ensuring accuracy and efficiency in data processing.


In [ ]:
vector_store.load_vector_store(settings.VECTOR_STORE_PATH)

2

In [28]:
r.shape

(2, 384)

In [5]:
if x:
    print("hi")

hi


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline
from langchain_huggingface import HuggingFacePipeline

In [4]:
model_id = "ALLaM-AI/ALLaM-7B-Instruct-preview"
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_4bit=True,
    #attn_implementation="flash_attention_2", # if you have an ampere GPU
)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=100, top_k=50, temperature=0.1)
llm = HuggingFacePipeline(pipeline=pipe)
llm.invoke("Hugging Face is")

PackageNotFoundError: No package metadata was found for bitsandbytes

In [7]:
messages = [
    {"role": "user", "content": "how to do coffee?"},
]
response = rag.generate_response(messages)
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] how to do coffee? [/INST] To make coffee, follow these steps:

1. Gather the ingredients and materials: coffee beans (ground or whole), water, a coffee maker, a filter, and a cup or mug.
2. Measure the coffee grounds: Depending on your preference, you can use 1 to 2 tablespoons of coffee grounds for every 6 ounces of water.
3. Add the coffee grounds to the filter: Place the filter in the coffee maker and pour the measured coffee grounds into it.
4. Add water: Fill the coffee maker with cold water, ensuring that it's within the water line.
5. Turn on the coffee maker: Allow the machine to heat the water and drip through the coffee grounds.
6. Wait for the coffee to brew: This process usually takes around 5-10 minutes, depending on the type of coffee maker.
7. Remove the coffee maker from the heat source: Once the brewing is complete, remove the coffee maker from the heat source.
8. Pour the coffee into a cup or mug: Carefully pour the freshly brewed coffee into your cup or mug.
9

In [8]:
response = rag.retrieve_and_generate("How much is the insurance pricing for someone ages less than 18 years old?")
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] <<SYS>>
You are a helpful AI assistant. Use the given context to answer the question.
<</SYS>>

Context:
Insurance Pricing Policy 
• Under 18 years old → 1,000 SR 
• Ages 19 - 30 → 1,200 SR 
• Ages 31 - 65 → 1,500 SR 
• Over 65 years old → 2,000 SR

Question: How much is the insurance pricing for someone ages less than 18 years old?

Answer: [/INST] The insurance pricing for someone ages less than 18 years old is 1,000 SR. 


In [13]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
response = rag.retrieve_and_generate("If there is 5 children less than 18 years old, and two adults between 31-65 years old, how much would be the final premium?")
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] <<SYS>>
You are a helpful AI assistant. Use the given context to answer the question.
<</SYS>>

Context:
Insurance Pricing Policy 
• Under 18 years old → 1,000 SR 
• Ages 19 - 30 → 1,200 SR 
• Ages 31 - 65 → 1,500 SR 
• Over 65 years old → 2,000 SR

Question: If there is 5 children less than 18 years old, and two adults between 31-65 years old, how much would be the final premium?

Answer: [/INST] Based on the provided Insurance Pricing Policy, we need to calculate the final premium for the given group of 5 children under 18 years old and 2 adults between 31-65 years old.

1. Calculate the premium for the 5 children under 18 years old:
   - Premium for one child = 1,000 SR
   - Premium for 5 children = 1,000 SR * 5 = 5,000 SR

2. Calculate the premium for the 2 adults between 31-65 years old:
   - Premium for one adult = 1,500 SR
   - Premium for 2 adults = 1,500 SR * 2 = 3,000 SR

3. Add the premiums for children and adults to get the final premium:
   - Total premium = 5,000 S

In [ ]:
query = "If there is 5 children less than 18 years old, and two adults between 31-65 years old, how much would be the final premium?"
response = rag.retrieve_and_generate_history(query, )
print(response)